In [ ]:
# Packages
suppressPackageStartupMessages({
  library(tidyverse)
  library(lubridate)
  library(MASS)
})

# --- Data path (prefer data/, fallback to root)
candidates <- c('data/btc_hourly_ohclv_ta.csv', 'btc_hourly_ohclv_ta.csv')
data_path <- candidates[file.exists(candidates)][1]
if (is.na(data_path)) stop('Could not find BTC CSV. Tried: ', paste(candidates, collapse = ', '))
message('Using: ', data_path)

# Load
df <- readr::read_csv(data_path, show_col_types = FALSE)

# Parse datetime and define Q
df <- df %>%
  mutate(DATETIME = ymd_hms(DATETIME, quiet = TRUE),
         Q = as.numeric(CLOSE)) %>%
  filter(!is.na(DATETIME), !is.na(Q)) %>%
  arrange(DATETIME)

message('Rows: ', nrow(df), ' Cols: ', ncol(df))
message('Datetime range: ', min(df$DATETIME), ' -> ', max(df$DATETIME))
df %>% select(DATETIME, Q) %>% head()

In [ ]:
# Helpers
ecdf_xy <- function(x) {
  x <- x[is.finite(x)]
  x <- sort(x)
  y <- seq_along(x) / length(x)
  list(x = x, y = y)
}

aic <- function(ll, k) 2 * k - 2 * ll
bic <- function(ll, k, n) k * log(n) - 2 * ll

skewness_simple <- function(x) {
  x <- x[is.finite(x)]
  m <- mean(x)
  s <- sd(x)
  if (!is.finite(s) || s == 0) return(NA_real_)
  mean(((x - m) / s)^3)
}

excess_kurtosis_simple <- function(x) {
  x <- x[is.finite(x)]
  m <- mean(x)
  s <- sd(x)
  if (!is.finite(s) || s == 0) return(NA_real_)
  mean(((x - m) / s)^4) - 3
}

summarize_series <- function(x) {
  x <- x[is.finite(x)]
  p <- as.numeric(stats::quantile(x, probs = c(0.01, 0.05, 0.25, 0.75, 0.95, 0.99), names = FALSE))
  tibble(
    n = length(x),
    min = min(x),
    max = max(x),
    mean = mean(x),
    median = median(x),
    sd = sd(x),
    skew = skewness_simple(x),
    excess_kurtosis = excess_kurtosis_simple(x),
    p01 = p[1],
    p05 = p[2],
    p25 = p[3],
    p75 = p[4],
    p95 = p[5],
    p99 = p[6]
  )
}

fit_norm <- function(x) {
  x <- x[is.finite(x)]
  mu <- mean(x)
  sigma <- sd(x)
  ll <- sum(dnorm(x, mean = mu, sd = sigma, log = TRUE))
  list(distribution = 'norm', params = list(mean = mu, sd = sigma), ll = ll, k = 2, n = length(x))
}

fit_lognorm <- function(x) {
  x <- x[is.finite(x) & x > 0]
  fit <- MASS::fitdistr(x, densfun = 'lognormal')
  ll <- as.numeric(logLik(fit))
  list(distribution = 'lognorm', params = as.list(fit$estimate), ll = ll, k = 2, n = length(x))
}

fit_gamma <- function(x) {
  x <- x[is.finite(x) & x > 0]
  fit <- MASS::fitdistr(x, densfun = 'gamma')
  ll <- as.numeric(logLik(fit))
  list(distribution = 'gamma', params = as.list(fit$estimate), ll = ll, k = 2, n = length(x))
}

ks_pvalue <- function(x, dist, params) {
  x <- x[is.finite(x)]
  out <- tryCatch({
    if (dist == 'norm') {
      stats::ks.test(x, 'pnorm', mean = params$mean, sd = params$sd)$p.value
    } else if (dist == 'lognorm') {
      stats::ks.test(x, 'plnorm', meanlog = params$meanlog, sdlog = params$sdlog)$p.value
    } else if (dist == 'gamma') {
      stats::ks.test(x, 'pgamma', shape = params$shape, rate = params$rate)$p.value
    } else {
      NA_real_
    }
  }, error = function(e) NA_real_)
  as.numeric(out)
}

## Task 1 — Distribution (Q = CLOSE)

We compute descriptive stats, fit candidate distributions (Normal / Lognormal / Gamma), compare them with log-likelihood + AIC/BIC, and visualize fit quality.

In [ ]:
Q <- df$Q
Q <- Q[is.finite(Q)]
Q_pos <- Q[Q > 0]

# --- Descriptive stats
stats_table <- summarize_series(Q)
stats_table

In [ ]:
# --- Fit and compare distributions
fits <- list(
  fit_norm(Q),
  fit_lognorm(Q_pos),
  fit_gamma(Q_pos)
)

comparison <- purrr::map_dfr(fits, function(f) {
  tibble(
    distribution = f$distribution,
    n_used = f$n,
    loglik = f$ll,
    AIC = aic(f$ll, f$k),
    BIC = bic(f$ll, f$k, f$n),
    KS_p = ks_pvalue(if (f$distribution == 'norm') Q else Q_pos, f$distribution, f$params)
  )
}) %>% arrange(AIC, BIC)

comparison

In [ ]:
# --- Task 1 plots (4)
fits_named <- purrr::set_names(fits, purrr::map_chr(fits, 'distribution'))

# 1) Histogram + fitted PDFs
ggplot(tibble(Q = Q_pos), aes(Q)) +
  geom_histogram(aes(y = after_stat(density)), bins = 120, fill = 'steelblue', alpha = 0.35) +
  stat_function(fun = dlnorm, args = fits_named$lognorm$params, linewidth = 1, colour = 'orange') +
  stat_function(fun = dgamma, args = fits_named$gamma$params, linewidth = 1, colour = 'green4') +
  stat_function(fun = dnorm, args = fits_named$norm$params, linewidth = 1, colour = 'red') +
  labs(title = 'Histogram + fitted PDFs', x = 'Q (CLOSE)', y = 'Density')

# 2) ECDF + fitted CDFs
xgrid <- seq(min(Q_pos), max(Q_pos), length.out = 2000)
ec <- ecdf(Q_pos)
ggplot() +
  geom_step(aes(xgrid, ec(xgrid)), linewidth = 1, colour = 'black') +
  geom_line(aes(xgrid, plnorm(xgrid, !!!fits_named$lognorm$params)), linewidth = 1, colour = 'orange') +
  geom_line(aes(xgrid, pgamma(xgrid, !!!fits_named$gamma$params)), linewidth = 1, colour = 'green4') +
  geom_line(aes(xgrid, pnorm(xgrid, !!!fits_named$norm$params)), linewidth = 1, colour = 'red') +
  labs(title = 'ECDF + fitted CDFs', x = 'Q (CLOSE)', y = 'F(Q)')

# 3) QQ plot: log(Q) vs Normal
logQ <- log(Q_pos)
qq <- qqnorm(logQ, plot.it = FALSE)
qq_df <- tibble(theoretical = qq$x, ordered = qq$y)
ggplot(qq_df, aes(theoretical, ordered)) +
  geom_point(size = 0.8, alpha = 0.7) +
  geom_abline(intercept = mean(logQ), slope = sd(logQ), colour = 'red') +
  labs(title = 'QQ plot: log(Q) vs Normal', x = 'Theoretical quantiles', y = 'Ordered values')

# 4) Density with log10 x-scale
ggplot(tibble(Q = Q_pos), aes(Q)) +
  geom_density(linewidth = 1, colour = 'steelblue') +
  scale_x_log10() +
  labs(title = 'Density with log10 x-scale', x = 'Q (log scale)', y = 'Density')

## Task 2 — Dry & Wet Years

We identify the 2 driest and 2 wettest years based on annual mean(Q), then compare intra-annual behavior (time series, cumulative, monthly distribution, monthly mean).

In [ ]:
d <- df %>% mutate(YEAR = year(DATETIME), MONTH = month(DATETIME))

annual <- d %>%
  group_by(YEAR) %>%
  summarise(mean_Q = mean(Q), n = n(), .groups = 'drop') %>%
  arrange(mean_Q)

selected_dry <- annual %>% slice_head(n = 2) %>% mutate(label = 'dry')
selected_wet <- annual %>% slice_tail(n = 2) %>% mutate(label = 'wet')
selected <- bind_rows(selected_dry, selected_wet) %>% arrange(label, mean_Q)
selected_years <- selected$YEAR
selected

In [ ]:
# --- Annual summary table (preview)
annual %>% head(10)

In [ ]:
# --- Seasonal signature table for selected years
sel <- d %>% filter(YEAR %in% selected_years)

seasonal_signature <- sel %>%
  group_by(YEAR, MONTH) %>%
  summarise(
    mean_Q = mean(Q),
    median_Q = median(Q),
    q25 = quantile(Q, 0.25),
    q75 = quantile(Q, 0.75),
    n = n(),
    .groups = 'drop'
  )
seasonal_signature %>% head(12)

In [ ]:
# --- Task 2 plots (4)

# 1) Time series (faceted by year)
ggplot(sel, aes(DATETIME, Q)) +
  geom_line(linewidth = 0.3) +
  facet_wrap(~ YEAR, scales = 'free_x', ncol = 2) +
  labs(title = 'Time series of Q (selected years)', x = 'Date', y = 'Q')

# 2) Annual cumulative sum within year
cum_df <- sel %>% arrange(YEAR, DATETIME) %>% group_by(YEAR) %>% mutate(cum_Q = cumsum(Q)) %>% ungroup()
ggplot(cum_df, aes(DATETIME, cum_Q, colour = factor(YEAR))) +
  geom_line(linewidth = 0.6) +
  labs(title = 'Annual cumulative sum of Q', x = 'Date', y = 'Cumulative Q', colour = 'Year')

# 3) Monthly boxplots
ggplot(sel, aes(factor(MONTH), Q, fill = factor(YEAR))) +
  geom_boxplot(outlier.shape = NA) +
  labs(title = 'Monthly distribution of Q (selected years)', x = 'Month', y = 'Q', fill = 'Year')

# 4) Monthly mean lines
monthly_means <- sel %>% group_by(YEAR, MONTH) %>% summarise(mean_Q = mean(Q), .groups = 'drop')
ggplot(monthly_means, aes(MONTH, mean_Q, colour = factor(YEAR))) +
  geom_line(linewidth = 0.8) +
  geom_point(size = 2) +
  scale_x_continuous(breaks = 1:12) +
  labs(title = 'Monthly mean of Q (selected years)', x = 'Month', y = 'Mean Q', colour = 'Year')